In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import re
from itertools import combinations
from tqdm.notebook import tqdm
import glob
import datetime

plt.style.use("dark_background")

path = "C:/Windows/Fonts/malgun.ttf"
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
mpl.rc("font", family=font_name)
mpl.rc("axes", unicode_minus=False)

In [18]:
# pref = "D:/디지털혁신팀/태블로/CS/CS 현황_FINAL/5월/"
# filenames = ["AS(20).csv", "AS(21).csv"]
# dfs = []
# for filename in filenames:
#     dfs.append(pd.read_csv(pref + filename, encoding="euc-kr"))
# as_list = pd.concat(dfs, ignore_index=True)
# # as_list.to_csv(f"{pref}AS리스트.csv", encoding="euc-kr")

# filenames = filenames = ["BS(19.6~19.12).csv", "BS(20).csv", "BS(21).csv"]
# dfs = []
# for filename in filenames:
#     dfs.append(pd.read_csv(pref + filename, encoding="euc-kr"))
# bs_list = pd.concat(dfs, ignore_index=True)
# bs_list = bs_list.rename({"아이앤콘스":"현장: 아이앤콘스"}, axis=1)

# # bs_list.to_csv(f"{pref}BS리스트.csv", encoding="euc-kr")

In [24]:
pref = "D:/디지털혁신팀/태블로/CS/CS 현황_FINAL/6월/"
filenames = ["20년 AS.csv", "21년 AS.csv"]
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(pref + filename, encoding="euc-kr"))
as_list = pd.concat(dfs, ignore_index=True)
as_list.to_csv(f"{pref}AS리스트.csv", encoding="euc-kr")

filenames = filenames = ["19년 BS.csv", "20년 BS.csv", "21년 BS.csv"]
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(pref + filename, encoding="euc-kr"))
bs_list = pd.concat(dfs, ignore_index=True)
bs_list = bs_list.rename({"아이앤콘스":"현장: 아이앤콘스"}, axis=1)

bs_list.to_csv(f"{pref}BS리스트.csv", encoding="euc-kr")

In [25]:
as_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753888 entries, 0 to 753887
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   AS 관리: No.      753888 non-null  int64  
 1   AS 접수일          753888 non-null  object 
 2   AS 완료일          726082 non-null  object 
 3   동               730440 non-null  object 
 4   호               730440 non-null  float64
 5   상태              753888 non-null  object 
 6   공용부/부대시설        22956 non-null   object 
 7   중공종             753883 non-null  object 
 8   소공종             753876 non-null  object 
 9   하자유형            753671 non-null  object 
 10  하자상세            753712 non-null  object 
 11  소요일.            753888 non-null  int64  
 12  처리완료            753888 non-null  int64  
 13  미처리             753888 non-null  int64  
 14  장기미처리           753888 non-null  int64  
 15  원시공업체           753540 non-null  object 
 16  현장: 아이앤콘스       753888 non-null  int64  
 17  현장: 현장명   

In [27]:
as_list[as_list["현장: 현장명"]=="청주가경IPARK 3단지"]

,AS 관리: No.,AS 접수일,AS 완료일,동,호,상태,공용부/부대시설,중공종,소공종,하자유형,하자상세,소요일.,처리완료,미처리,장기미처리,원시공업체,현장: 아이앤콘스,현장: 현장명,현장: 현장코드 (ERP),현장: 입주일
737644,2419759,2021. 6. 11.,2021. 6. 18.,NaN,NaN,최종완료,홈오토 공용부(공용부),사인몰,사인물,손상,307동 5~6라인 지상1층 동출입구 캐노피 상부 5~6라인 숫자 싸인몰파손(6/1...,7,1,0,0,대명아트㈜,0,청주가경IPARK 3단지,C031,2021. 5. 31.
737645,2411598,2021. 6. 5.,2021. 6. 7.,307,503.0,최종완료,NaN,타일,타일,줄눈불량,현관문틀과 바닥 타일사이에 일부가 실리콘이 시공되지 않음A/S 되지 않았는데 처리 ...,2,1,0,0,진명건설㈜,0,청주가경IPARK 3단지,C031,2021. 5. 31.
737646,2403985,2021. 5. 31.,2021. 5. 31.,NaN,NaN,최종완료,홈오토공용부(공용계단실),골조,골조,균열,맘스클럽 바닥 걸레받이와 디럭스 타일사이 틈새,0,1,0,0,윤주건설㈜,0,청주가경IPARK 3단지,C031,2021. 5. 31.
737647,2429177,2021. 6. 20.,2021. 6. 23.,303,301.0,최종완료,NaN,등기구,등기구,작동불량,거실 아트월 맞은편 간접조명 두개 중 하나가 안 들어옵니다_인테리어세대,3,1,0,0,에이치디씨아이콘트롤스(주),0,청주가경IPARK 3단지,C031,2021. 5. 31.
737648,2419738,2021. 6. 11.,2021. 6. 16.,NaN,NaN,최종완료,홈오토 공용부(공용부),석공사,바닥석재,균열,307동 5~6라인 지상1층 동출입구 캐노피 상부 대리석2~3장 파손(6/11 세대...,5,1,0,0,에이치디씨영창(주),0,청주가경IPARK 3단지,C031,2021. 5. 31.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747507,2423090,2021. 6. 14.,2021. 6. 20.,305,1601.0,최종완료,NaN,코킹,코킹,코킹불량,욕조와 바닥타일 사이 코킹 들뜸,6,1,0,0,㈜예찬도장기업,0,청주가경IPARK 3단지,C031,2021. 5. 31.
747508,2423104,2021. 6. 14.,2021. 6. 30.,305,1601.0,최종완료,NaN,온돌마루,온돌마루,손상,분합창 앞 마루 긁힘 1장,16,1,0,0,(주)윈앤윈우드,0,청주가경IPARK 3단지,C031,2021. 5. 31.
747509,2425758,2021. 6. 16.,2021. 6. 21.,305,1601.0,최종완료,NaN,목창호,목창호 문틀,손상,안방 드레스룸 들어가는 문틀(욕실문 옆 문틀) 중간부분 손상\n6/21.1~4시방문예정,5,1,0,0,주식회사 우디스,0,청주가경IPARK 3단지,C031,2021. 5. 31.
747510,2434947,2021. 6. 25.,NaN,305,1601.0,접수,NaN,손끼임방지,손끼임방지,손상,[예약: 6/29 화 오전] 문 계폐 안전장치 래핑 불량여부 확인바람.6/29PL창...,6,0,1,0,（주）아이손,0,청주가경IPARK 3단지,C031,2021. 5. 31.
